## TO-DO List

**1) Today's tasks** - select 1 to display all today's tasks <br/>
**2) Week's tasks** - select 2 to display all tasks in the upcoming week <br/>
**3) All tasksv** - select 3 to display all available task <br/>
**4) Missed tasks** - select 4 to display all tasks, which assigned date has already passed <br/>
**5) Add Task** - select 5 to add new task <br/>
**6) Delete task** - select 6 to delete task (enter date in format yyyy-mm-dd) <br/>
**0) Exit** - select 0 to exit To-Do list <br/>

In [ ]:
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///todo.db?check_same_thread=False')
Base = declarative_base()

class Table(Base):
    __tablename__ = 'task'
    id = Column(Integer, primary_key=True)
    task = Column(String)
    deadline = Column(Date, default=datetime.today())
    def __repr__(self):
        return self.task

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

a = '4'
while a != '0':
    print("\n1) Today's tasks\n2) Week's tasks\n3) All tasks\n4) Missed tasks\n5) Add Task\n6) Delete task\n0) Exit")
    a = input()
    if a == '1':
        rows = session.query(Table).filter(Table.deadline==datetime.today().date()).all()
        print('\nToday', datetime.today().strftime('%#d %b'),':')
        if rows == []:
            print('Nothing to do!')
        else:
            # first_row = rows[0] # In case rows list is not empty
            for i,n in enumerate(rows):
                print(i+1, ". ",n.task)
    elif a == '2':
        rows = session.query(Table).all()
        for j in range(7):
            day = (datetime.today().date() + timedelta(days=j))
            rows = session.query(Table).filter(Table.deadline==day).all()
            print("\n")
            print(day.strftime('%A %#d %b'), ":")
            if rows == []:
                print('Nothing to do!')
            else:
                for i,n in enumerate(rows):
                    print(i+1, ". ",n.task)
    elif a == '3':
        print("\nAll tasks")
        rows = session.query(Table).order_by(Table.deadline).all()
        if rows == []:
            print('Nothing to do!')
        else:
            # first_row = rows[0] # In case rows list is not empty
            for i,n in enumerate(rows):
                print(f"{i+1}. {n.task}. {n.deadline.strftime('%#d %b')}")
    elif a == '4':
        rows = session.query(Table).filter(Table.deadline<datetime.today().date()).order_by(Table.deadline).all()
        print("\nMissed tasks:")
        if rows == []:
            print('Nothing is missed!')
        else:
            # first_row = rows[0] # In case rows list is not empty
            for i,n in enumerate(rows):
                print(f"{i+1}. {n.task}. {n.deadline.strftime('%#d %b')}")
    elif a == '5':
        print('Enter task')
        a = input()
        print("Enter deadline")
        b = input()
        new_row = Table(task=a, deadline = datetime.strptime(b, "%Y-%m-%d"))
        session.add(new_row)
        session.commit()
        print('The task has been added!')
    elif a == '6':
        print("\nChose the number of the task you want to delete:")
        rows = session.query(Table).order_by(Table.deadline).all()
        if rows == []:
            print('Nothing to delete')
        else:
            for i,n in enumerate(rows):
                print(f"{i+1}. {n.task}. {n.deadline.strftime('%#d %b')}")
        specific_row = rows[int(input())-1]
        session.delete(specific_row)
        session.commit()
    elif a =='0':
        print('\nBye!')